In [46]:
from data4ml import Data4ML
from bs4 import BeautifulSoup
import json
import os
import re
from perfect_regex import (
    perfect_url_regex,
    perfect_emoji_regex,
    perfect_email_regex,
    perfect_phone_regex,
)

In [39]:
class Data4Chatbot(Data4ML):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def make_data(self):
        pass
    
    def parse_html(self, parent_folder: str, files: list) -> list:
        pass
    
    def clear_message(self, i: str) -> list:
        assert isinstance(i, str)
        
        cleared_messages = []
        # If `Ссылка` in message - not append this message:
        if "\nСсылка\nhttps:" in i or "#comments" in i:
            return ''

        # Delete trash such as stickers, attached messages:
        for end in self.message_ends:
            if i.endswith(end):
                i = i[: i.rfind("\n")]

        # Delete attachments such as photos, documents, ect.:
        for attachment in self.blacklist:
            i = re.sub(f"[\n]?{attachment}[\n]?" + perfect_url_regex, "", i)
            i = re.sub(f"[\n]?{attachment}[\n]?$", "", i)

        # Delete trash:
        i = re.sub(perfect_emoji_regex, "", i)
        i = re.sub(perfect_email_regex, "", i)
        i = re.sub(perfect_phone_regex, " ", i)
        i = re.sub(perfect_url_regex, "", i)
        i = re.sub(f"[\n]?Аудиозапись[\n]?", "", i)

        i = i.replace("  ", " ")  # remove double space
        i = i.strip()

        if i:
            return i

In [40]:
chatbot = Data4Chatbot(path_to_config='./../data_params.json')

In [41]:
list_of_folders = chatbot.get_list_of_folders('data4test')
print(list_of_folders)

['.ipynb_checkpoints', '153164713', '153164714']


In [42]:
list_of_files = chatbot.get_list_of_files_in_folder('data4test/153164714')
print(list_of_files)

['messages0.html']


In [43]:
chatbot.parse_html('data4test/153164714', list_of_files)

In [44]:
with open(os.path.join('data4test/153164714', 'messages0.html'), "rb") as f:
    soup = BeautifulSoup(f, "lxml")

In [60]:
def read_mes(parent_folder, files):
    all_messages = []
    for file in files:
        with open(os.path.join(parent_folder, file), "rb") as f:
            soup = BeautifulSoup(f, "lxml")

        messages = []
        for message in soup.find_all("div", {"class": "message"}):
            message = message.text.strip()
            messages.append(message)

        # Reverse the list to save the message sequence:
        all_messages.extend(messages[::-1])
        
    return all_messages

In [62]:
all_messages = read_mes('data4test/153164714', ['messages0.html'])

In [70]:
messages = []

message_one = ""
last_autor = None
is_first = True

for message in all_messages:
    autor = message[:message.find(',')]
        
    message = message[message.find("\n") + 1 :]
    clear_message = chatbot.clear_message(message)
    if not clear_message:
        continue
    
    if autor != last_autor: # new interlocutor
        messages.append(autor + ': ' + message_one)
        message_one = ''
        
    message_one += clear_message
    message_one += '.'


In [71]:
messages

['Юлия Николаева: ',
 'Вы: пришли фотки, пожалуйста.',
 'Вы: Привет, сейчас.',
 'Юлия Николаева: Всё👌.',
 'Вы: Спасибо большое.',
 'Юлия Николаева: 👌.',
 'Юлия Николаева: Сережка, перешли мне письмо Смоленцевой..',
 'Вы: Заранее спасибо.',
 'Юлия Николаева: Какое именно?.',
 'Юлия Николаева: Последнее. Про кино.',
 'Вы: Спасибо.',
 'Вы: Скинул.',
 'Юлия Николаева: Подойдите завтра к смоленцевой.',
 'Вы: зачем?.',
 'Юлия Николаева: Она попросила.',
 'Вы: Хорошо.',
 'Вы: Юль.',
 'Вы: не забывай, что вы завтра в 12 танцуете.',
 'Юлия Николаева: И своим передай.',
 'Юлия Николаева: Хорошо.',
 'Юлия Николаева: спасибо.',
 'Юлия Николаева: Спасибо.',
 'Юлия Николаева: а до скольких?.']